In [1]:
cd C:\Users\yulya\PycharmProjects\machine_learning_of_patterns

C:\Users\yulya\PycharmProjects\machine_learning_of_patterns


In [2]:
import time
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
from random import randint, random
from typing import Callable, Any, Tuple, List, Optional, Dict, Iterator
from tqdm import tqdm

In [3]:
from regex.parser import RegexParser

In [4]:
"""
GRAMMAR:
    <regex> ::= <conc-regex> <alt> <regex> | <conc-regex>
    <conc-regex> ::= <simple-regex> | <simple-regex><conc-regex>
    <simple-regex> ::= <lbr><regex><rbr><unary>? | letter <unary>?
    <alt> ::= "|"
    <lbr> ::= "("
    <rbr> ::= ")"
    <unary> ::= "*"
"""


class RegexGenerator:
    seed = 0
    cur_regex_length = 0
    cur_star_num = 0
    cur_nesting = 0
    cur_regex = ""
    count = -1

    def __init__(
            self, regex_length: int,
            star_num: int, star_nesting: int,
            alphabet_size: int, alphabet: Optional[List[str]] = None):
        assert regex_length >= 1, "regex length must be > 0"
        self.regex_length = regex_length
        self.star_nesting = 0 if star_nesting < 0 else star_nesting
        self.star_num = 0 if star_num < 0 else star_num
        if alphabet is None:
            if alphabet_size < 1:
                alphabet_size = 1
            self.alphabet = list(string.ascii_lowercase)[:alphabet_size] + \
                            list(string.ascii_uppercase)[:alphabet_size]
        else:
            self.alphabet = alphabet

    def change_seed(self, step: int = 1):
        self.seed += step
        # random.seed(self.seed)

    def generate_regex(self) -> str:
        # self.change_seed()
        self.cur_nesting = 0
        self.cur_regex = ""
        self.cur_regex_length = self.regex_length
        self.cur_star_num = self.star_num
        self.__generate_regex()
        return self.cur_regex

    # <regex> ::= <n-alt-regex> <alt> <regex> | <conc-regex>
    def __generate_regex(self):
        if self.cur_regex_length < 1:
            return
        if random.randint(0, 1):
            if self.cur_regex_length > 1:
                self.generate_conc_regex()
            if self.cur_regex_length < 1:
                return
            if self.cur_regex_length != 1:
                self.cur_regex += "|"
                self.__generate_regex()
        else:
            self.generate_conc_regex()

    # <conc-regex> ::= <simple-regex> | <simple-regex><conc-regex>
    def generate_conc_regex(self):
        if random.randint(0, 1):
            self.generate_simple_regex()
            if self.cur_regex_length < 1:
                return
            self.generate_conc_regex()
        else:
            self.generate_simple_regex()

    # <simple-regex> ::= <lbr><regex><rbr><unary>? | letter <unary>?
    def generate_simple_regex(self, br_prob: int = 7):
        if random.randint(0, 10) > br_prob:
            self.cur_regex += self.random_char()
            if self.cur_star_num:
                star_chance = self.cur_regex_length // self.cur_star_num
                if star_chance < 2:
                    star_chance = 2
                v = random.randint(0, star_chance - 1)
            else:
                v = 1
            if not v and self.cur_star_num > 0 and self.cur_nesting < self.star_nesting:
                self.cur_regex += "*"
                self.cur_star_num -= 1
            self.cur_regex_length -= 1
        else:
            if self.cur_star_num:
                star_chance = self.cur_regex_length // self.cur_star_num
                if self.cur_regex_length > self.cur_star_num:
                    star_chance += self.cur_star_num // self.star_nesting
                else:
                    star_chance += self.cur_regex_length // self.star_nesting
                if star_chance < 2:
                    star_chance += 2
                v = random.randint(0, star_chance - 1)
            else:
                v = 1
            if not v and self.cur_star_num > 0 and self.cur_nesting < self.star_nesting:
                self.cur_star_num -= 1
                self.cur_nesting += 1
            else:
                v = 1
            self.cur_regex += "("
            self.__generate_regex()
            self.cur_regex += ")"
            if not v:
                self.cur_regex += "*"
                self.cur_nesting -= 1

    def random_char(self):
        return self.alphabet[random.randint(0, len(self.alphabet) - 1)]

In [75]:
def generate_stateful_regex(
        length_range: List[int],
        star_num: int, 
        star_nesting: int, 
        alphabet_size: int = 26,
        alphabet: Optional[List[str]] = None) -> str:
    gen = RegexGenerator(length_range[1], star_num, star_nesting, alphabet_size, alphabet)
    regex = ""
    while not (length_range[0] <= len(regex) <= length_range[1]):
        regex = gen.generate_regex()
    return regex


def measure_time(func: Callable, *params) -> Tuple: # in seconds
    t1 = time.perf_counter()
    result = func(*params)
    t2 = time.perf_counter()
    return result, t2 - t1


def count_items_in_regex(regex: str, alphabet_size: int = 26) -> Tuple[int]:
    stars = 0
    letters = 0
    alphabet = list(string.ascii_lowercase)[:alphabet_size] + \
               list(string.ascii_uppercase)[:alphabet_size]
    for c in regex:
        if c in alphabet:
            letters += 1
        elif c == '*':
            stars += 1
    return letters, stars


def get_letters(word: str) -> List[str]:
    letters = []
    alphabet = list(string.ascii_lowercase) + \
               list(string.ascii_uppercase)
    for c in word:
        if c in alphabet:
            letters.append(c)
    return letters


def get_chars(word: str) -> List[str]:
    return sorted(list(set(word)))


def generate_word_group(
    step: int = 10, max_n: int = 50, 
    length_range: List[int] = [3, 30],
    alphabet: Optional[List[str]] = None) -> List[str]:
    if alphabet is None:
        alphabet = list(string.ascii_lowercase + string.ascii_uppercase)
    alphabet_len = len(alphabet)
    pattern = []
    mask = []
    length = randint(length_range[0], length_range[1])
    for i in range(length):
        if i < len(alphabet):
            pattern.append(alphabet[i])
        else:
            pattern.append(alphabet[randint(0, alphabet_len - 1)])
        mask.append(randint(0, 1))
    return ("".join(pattern), 
            ["".join([p * n if mask[i] else p for i, p in enumerate(pattern)]) for n in range(0, max_n + 1, step)])


def detect_exp(
        length: List[int], 
        time: List[float], 
        c1_range: Iterator[int], 
        c2_range: Iterator[int]) -> bool:
    for c1 in c1_range:
        for c2 in c2_range:
            exp = False
            for l, t in zip(length, time):
                if t >= 10**c1 * l**4 + 10**c2:
                    exp = True
                else:
                    exp = False
                    break
            if exp:
                return True
    return False


def plot_dependence(df: pd.DataFrame):
    plt.figure(figsize=(10, 5))
    xs = df['length'][::3] 
    types = df['matching_type'].unique()
    for label in types:
        plt.plot(xs, df[df['matching_type'] == label]['time'], label=label)
    plt.ylabel('Time, sec')
    plt.xlabel('Step')
    plt.legend()
    plt.show();

In [76]:
def test(
        generate_word_group: Callable, 
        step: int = 10, max_n: int = 50,
        plot: bool = False) -> pd.DataFrame:
    regex_str = generate_stateful_regex([15, 50], 20, 40, alphabet=None)
    alphabet = get_letters(regex_str)
    pattern, words = generate_word_group(step, max_n, alphabet=alphabet)
    print(pattern)
    print(regex_str)
    print(words[1])
    regex = RegexParser(regex_str, [max_n ** 2] * regex_str.count('*')).parse()
    nfa = regex.to_nfa()
    dfa = nfa.to_dfa()

    data = {
        "regex": regex,
        "nfa": nfa,
        "dfa": dfa
    }
    results = {
        "length": [],
        "matching_type": [],
        "time": [],
        "match": []
    }
    
    for word in words:
        for k, v in data.items():
            match, t = measure_time(v.match, word)
            results["length"].append(len(word))
            results["matching_type"].append(k)
            results["time"].append(t)
            results["match"].append(match)
            
    df = pd.DataFrame(data=results)
    if plot:
        plot_dependence(df)
    return words, regex_str, df

### Testing

In [77]:
data={"regex": [], "is_target": []}
all_words = []

for i in range(300):
    words, regex, result = test(generate_word_group, 10, 100)
    all_words.append(words)
    data["regex"].append(regex)
    is_exp = False
    for name in ["regex", "nfa", "dfa"]:
        is_exp = detect_exp(
            result[result['matching_type'] == name]["length"], 
            result[result['matching_type'] == name]["time"], 
            range(2, -10, -1), range(2, -10, -1))
        if is_exp:
            break
    data["is_target"].append(is_exp)
    
df = pd.DataFrame(data=data)

IuLzKJuzIuIIJzuLIKuIILJ
I(u)L*(((z)K*(J)*)*)
IuLLLLLLLLLLzzzzzzzzzzKJJJJJJJJJJuuuuuuuuuuzIIIIIIIIIIuuuuuuuuuuIIIIIIIIIIIIIIIIIIIIJzuuuuuuuuuuLIIIIIIIIIIKuIIIIIIIIIIILLLLLLLLLLJ
IHHFxFFIFFFFxFx
((I*)*(H*)H)*((F|x*))*|FF*
IHHHHHHHHHHHHHHHHHHHHFxxxxxxxxxxFFFFFFFFFFFFFFFFFFFFIIIIIIIIIIFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFxFx
PUhIA
((P)(U*))((h)*I|A*)
PPPPPPPPPPUUUUUUUUUUhhhhhhhhhhIA
dwkTzTzwkdTwT
(((d*|(wk*)*T*)*|((z*)*)))
ddddddddddwkkkkkkkkkkTzzzzzzzzzzTTTTTTTTTTzwwwwwwwwwwkddddddddddTTTTTTTTTTwwwwwwwwwwT
GrvyNNrGNvGGrGrrGGyvGrN
Gr*|(v((y*)*)*)*|N
GrrrrrrrrrrvyNNNNNNNNNNNNNNNNNNNNrGNNNNNNNNNNvvvvvvvvvvGGGGGGGGGGGrGrrrrrrrrrrrGGGGGGGGGGGGGGGGGGGGyvvvvvvvvvvGGGGGGGGGGrN
eXCwoWrtwCWeoX
(((eX*(Cw)o*)W*)*|r)*|t*
eeeeeeeeeeXXXXXXXXXXCwwwwwwwwwwoWWWWWWWWWWrttttttttttwCCCCCCCCCCWWWWWWWWWWeoXXXXXXXXXX
lmwQQQ
l*((((m*|w*)|Q*))*)
lmmmmmmmmmmwQQQ
bGenT
b*|(G)(e)*(n*)*T*y
bbbbbbbbbbGeeeeeeeeeenT
nKihVhnhiKhiihhKii
((n*|((K*(i))|h*)*)V*)*
nnnnnnnnnnKihVhnhiiiiiiiiiiKKKKKKKKKKhiiiiiiiiiiihhhhhhhhhhhhhhhhhhhh

NfTrxTrrxrrfNrTrxNNrrxxTrTrfx
(((N*)*)*(f|T*)*)r(x)
NNNNNNNNNNffffffffffTTTTTTTTTTrrrrrrrrrrxTrrrrrrrrrrrrrrrrrrrrxxxxxxxxxxrrrrrrrrrrrfNNNNNNNNNNrTTTTTTTTTTrxxxxxxxxxxNNNNNNNNNNNrrrrrrrrrrrxxxxxxxxxxxTTTTTTTTTTrTTTTTTTTTTrrrrrrrrrrfx
bpMMMMbppMMbbbMMMpbMpM
(b*((p|(M*)))*)*
bppppppppppMMMMbppppppppppppppppppppMMMMMMMMMMMbbbbbbbbbbbbbbbbbbbbbMMMMMMMMMMMMpbMMMMMMMMMMpMMMMMMMMMM
CCPteiePeCCCtieeiPiP
C*((C*|P)*)t|e|i
CCCCCCCCCCCCCCCCCCCCPPPPPPPPPPtttttttttteeeeeeeeeeiePPPPPPPPPPeeeeeeeeeeCCCCCCCCCCCCttttttttttieeeeeeeeeeeiPPPPPPPPPPiiiiiiiiiiPPPPPPPPPP
VghyU
(V*)(g*(hy)*UP)*
VVVVVVVVVVgggggggggghyyyyyyyyyyU
aalalalalaaaaalal
(((a*)*((al)*)))*
aaaaaaaaaaaaaaaaaaaallllllllllalaaaaaaaaaallllllllllalaaaaaaaaaaaaaallllllllllaaaaaaaaaallllllllll
aHnea
(a)*(H*|n*|e*)*
aaaaaaaaaaHHHHHHHHHHnea
JqiqiqJqiJqi
J*|(((q(i))*)*q)*
JJJJJJJJJJqqqqqqqqqqiiiiiiiiiiqiiiiiiiiiiqqqqqqqqqqJJJJJJJJJJqiiiiiiiiiiJqiiiiiiiiii
LOvNVvVVLLLVLNNvVONONNOvVONL
(((L*(O*))v)*)N*|((V)*)*
LLLLLLLLLLOvvvvvvvvvvNNNNNNNNNNVvvvvvv

KeyboardInterrupt: 

In [78]:
df = pd.DataFrame(data=data)

In [79]:
df

,regex,is_target
0,I(u)L*(((z)K*(J)*)*),False
1,((I*)*(H*)H)*((F|x*))*|FF*,False
2,((P)(U*))((h)*I|A*),False
3,(((d*|(wk*)*T*)*|((z*)*))),False
4,Gr*|(v((y*)*)*)*|N,False
...,...,...
125,(yA((Ir)))*((A*)*),False
126,p|k|Ha*|(J((i|g*)v))(l)*,False
127,((a*)((h)*)J*)t,False
128,(No*((t*b*)))*n,False


In [80]:
df[df["is_target"] == True]

,regex,is_target
78,(((f)*k|(r*)*(P*)*)*|M)*,True
87,(((KP))*(((i*))*)*|T|(S*)),True


In [89]:
dataset.to_csv("experiments/dataset.csv", index=False)

In [81]:
dataset = pd.concat([dataset, df[df["is_target"] == True]], ignore_index=True)

In [82]:
dataset.to_csv("dataset.csv", index=False)

In [83]:
dataset

,regex,is_target
0,l*l*l*,True
1,Z(J)*,True
2,k*|(s),True
3,Q*(((Q)|(Q*)*Q)*Q*),True
4,(C*)*,True
...,...,...
76,(J)((d*)*)*,True
77,x*h((Q*|P))*,True
78,(Y*|(m*))*,True
79,(((f)*k|(r*)*(P*)*)*|M)*,True
